In [1]:
import chess
import chess.engine
import random
import numpy as np
import torch

In [2]:
from generate_data import random_board, split_dims

sample_board = random_board()

# Get the 3D representation using split_dims
sample_board_representation = split_dims(sample_board)

# Print the shape
print("Output shape of split_dims:", sample_board_representation.shape)

import numpy as np
from torch.utils.data import Dataset

class ChessDataset(Dataset):
    def __init__(self, file_path):
        container = np.load(file_path, allow_pickle=True)
        self.b = container['b']
        self.v = np.asarray(container['v'] / abs(container['v']).max() / 2 + 0.5, dtype=np.float32)

    def __len__(self):
        return len(self.b)

    def __getitem__(self, idx):
        return self.b[idx], self.v[idx]

# Usage example
chess_dataset = ChessDataset('dataset.npz')
dataset_size = len(chess_dataset)
print(f"Dataset size: {dataset_size}")



Output shape of split_dims: (14, 8, 8)
Dataset size: 3743591


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, conv_size):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=conv_size, out_channels=conv_size, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(conv_size)
        self.conv2 = nn.Conv2d(in_channels=conv_size, out_channels=conv_size, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(conv_size)

    def forward(self, x):
        residual = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += residual
        out = F.relu(out)
        return out

class ResidualChessModel(nn.Module):
    def __init__(self, conv_size, conv_depth):
        super(ResidualChessModel, self).__init__()
        self.initial_conv = nn.Conv2d(in_channels=14, out_channels=conv_size, kernel_size=3, padding=1)
        self.residual_layers = nn.Sequential(*[ResidualBlock(conv_size) for _ in range(conv_depth)])
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(8 * 8 * conv_size, 1)

    def forward(self, x):
        x = F.relu(self.initial_conv(x))
        x = self.residual_layers(x)
        x = self.flatten(x)
        x = torch.sigmoid(self.fc(x))
        return x

class ChessConvNet(nn.Module):
    def __init__(self, conv_size, conv_depth):
        super(ChessConvNet, self).__init__()
        # Define the convolutional layers
        self.convs = nn.Sequential(
            *[
                nn.Conv2d(in_channels=conv_size if i > 0 else 1, 
                          out_channels=conv_size, 
                          kernel_size=3, 
                          padding='same') for i in range(conv_depth)
            ]
        )
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(conv_size * 14 * 8 * 8, 64)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.convs(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.sigmoid(x)
        return x
        
model = ResidualChessModel(conv_size=16, conv_depth=6)
model.load_state_dict(torch.load('model_weights.pth'))
model  = model.to('cuda')

In [4]:
squares_index = {
    'a' : 0,
    'b' : 1,
    'c' : 2,
    'd' : 3,
    'e' : 4,
    'f' : 5,
    'g' : 6,
    'h' : 7
}

def square_to_index(square):
    letter = chess.square_name(square)
    return 8 - int(letter[1]), squares_index[letter[0]]

def split_dims(board):
    board3d = np.zeros((14,8,8))
    
    for piece in chess.PIECE_TYPES:
        for square in board.pieces(piece, chess.WHITE):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece - 1][7 - idx[0]][idx[1]] = 1
        for square in board.pieces(piece, chess.BLACK):
            idx = np.unravel_index(square, (8, 8))
            board3d[piece + 5][7 - idx[0]][idx[1]] = 1
        
    aux = board.turn
    
    board.turn = chess.WHITE
    for move in board.legal_moves:
        i,j = square_to_index(move.to_square)
        board3d[12][i][j]
        
    board.turn = chess.BLACK
    for move in board.legal_moves:
        i,j = square_to_index(move.to_square)
        board3d[13][i][j]
    
    board.turn = aux
    
    return board3d

In [5]:
def minimax_eval(board):
    
    board3d = split_dims(board)
    board3d = torch.from_numpy(board3d)
    board3d = board3d.float()
    board3d = torch.unsqueeze(board3d, 0)
    board3d = board3d.to('cuda')
    
    with torch.no_grad():
        result = model(board3d)
    
    return result.cpu().detach().numpy()

def order_moves(board):
    captures = [move for move in board.legal_moves if board.is_capture(move)]
    non_captures = [move for move in board.legal_moves if not board.is_capture(move)]
    return captures + non_captures

def minimax(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or board.is_game_over():
        return minimax_eval(board)

    moves = order_moves(board)
    
    max_eval = - np.inf 
    min_eval =   np.inf 
    fraction_of_legal_moves = 1.0 
    
    if maximizing_player:    
        legal_moves = list(moves)
        n_legal_moves = len(legal_moves)
        
        for move in legal_moves[:int(fraction_of_legal_moves*n_legal_moves)]:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, False)
            board.pop( )
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    
    else:     
        legal_moves = list(moves)
        n_legal_moves = len(legal_moves) 
        
        for move in legal_moves[:int(fraction_of_legal_moves*n_legal_moves)]:
            board.push(move)
            eval = minimax(board, depth - 1, alpha, beta, True)
            board.pop( )
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break  
              
        return min_eval

def get_ai_move(board, depth):
    
    max_move = None
    max_eval = -np.inf
    
    legal_moves = [move for move in board.legal_moves]
    legal_moves = random.sample(legal_moves, int(len(legal_moves)*1.0))
    
    for move in board.legal_moves:
        board.push(move)
        eval = minimax(board, depth-1, -np.inf, np.inf, False)
        board.pop()
        if eval > max_eval:
            max_eval = eval
            max_move = move

    return max_move    

In [6]:
import chess.svg
from IPython.display import display, clear_output
import time

board = chess.Board()
size=600
wait_time = 0.5

with chess.engine.SimpleEngine.popen_uci('/home/oaltuner/Documents/chess/bora_hoca_kodlar/stockfish/stockfish-ubuntu-x86-64') as engine:
    
    while True:
        move = get_ai_move(board, 4)
        board.push (move)
        
        clear_output(wait=True)
        display(chess.svg.board(board=board, size=size))
        time.sleep(wait_time)
        
        if board.is_game_over():
            print('Check mate! White (AI) wins!')
            break
        
        result = engine.play(board, chess.engine.Limit(depth=4))
        board.push(result.move)
        
        clear_output(wait=True)
        display(chess.svg.board(board=board, size=size))
        time.sleep(wait_time)
        
        if board.is_game_over():
            print('Check mate! Black (Stockfish) wins!')
            break